### [문제]
- 세션ID 생성
- 세션별 대화 기록을 store에 임시 저장 : {}
- ChatPromptTemplate() : 역할 부여
- InMemoryChatMessageHistory() 생성
- RunnableWithMessageHistory() 생성
- chain 구성

### [문제] 기존 String PromptTemplate을 ChatPromptTemplate으로 변경
- Chain
- prompt template: 직접 작성
                   문서를 참고해서, 사용자 질문에 답변

In [70]:
#1. 모듈(파일, 라이브러리) 로드
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from dotenv import load_dotenv

#2. 환경변수 로드
load_dotenv()

#3. 파일 로드
file_name = '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'
with open(file_name, 'r', encoding='utf-8') as file:
    law = file.read()
    print(law[:300])
    
#4. 프롬프트템플릿 생성
template = '''다음 문서를 참고하여, 사용자 질문에 답변합니다. \n 문서 내용을 기반으로 답변하고, 해당 조항도 표시합니다. 
-문서: {law}
-질문: {query}
-히스토리: {chat_history}
'''
prompt = ChatPromptTemplate.from_template(template)

#5. 챗오픈ai 모델 생성    
llm = ChatOpenAI(
    model='gpt-4.1-nano', streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

prompt


남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률 ( 약칭: 남녀고용평등법 )
[시행 2025. 2. 23.] [법률 제20521호, 2024. 10. 22., 일부개정]

고용노동부(여성고용정책과) 044-202-7476

       제1장 총칙 <개정 2007. 12. 21.>
제1조(목적) 이 법은 「대한민국헌법」의 평등이념에 따라 고용에서 남녀의 평등한 기회와 대우를 보장하고 모성 보호와 여성 고용을 촉진하여 남녀고용평등을 실현함과 아울러 근로자의 일과 가정의 양립을 지원함으로써 모든 국민의 삶의 질 향상에 이바지하는 것을


ChatPromptTemplate(input_variables=['chat_history', 'law', 'query'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['chat_history', 'law', 'query'], input_types={}, partial_variables={}, template='다음 문서를 참고하여, 사용자 질문에 답변합니다. \n 문서 내용을 기반으로 답변하고, 해당 조항도 표시합니다. \n-문서: {law}\n-질문: {query}\n-히스토리: {chat_history}\n'), additional_kwargs={})])

In [69]:
#6. 체인 구성
chain = prompt | llm

#7. 은 히스토리 변수 지정인데 굳이 불필요할듯


#8. 함수 정의- 대화이력(히스토리)인스턴스 리턴
store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
	if session_id not in store :
		store[session_id] = InMemoryChatMessageHistory()
	return store[session_id]

#9. 질문 받기 
query = input('질문을 입력하세요...')

#10. 체인에 대화 히스토리 기능 래핑
wr_chain = RunnableWithMessageHistory(
	chain,
	get_session_history,
    input_messages_key='query',
	history_messages_key='chat_history'
	)

#11. 실행
wr_chain.invoke(
	{'law': law, 'query': query},
	config={'configurable':{'session_id':'솔라나'}}
)

네, 남성 근로자도 육아휴직을 신청할 수 있습니다. (제19조(육아휴직) 및 제19조의4(육아휴직 분할 사용))

AIMessage(content='네, 남성 근로자도 육아휴직을 신청할 수 있습니다. (제19조(육아휴직) 및 제19조의4(육아휴직 분할 사용))', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45'}, id='run--a9935c11-848c-4189-9dcf-6efc62002b7a-0')

In [57]:
# prompt = PromptTemplate.from_template(template=template) =====> 😉휴먼메시지,ai메시지 분리없이 드감 
# prompt = ChatPromptTemplate.from_messages(template)  ====> 😫에러이실게요~!
# prompt

In [58]:
len(store)

2

In [59]:
get_session_history('솔라나')

InMemoryChatMessageHistory(messages=[HumanMessage(content='육아휴직은 남성도 쓸 수 있나?', additional_kwargs={}, response_metadata={}), AIMessage(content='네, 육아휴직은 남성 근로자도 사용할 수 있습니다. 남녀고용평등법 제19조(육아휴직)에 따르면, 사업주는 임신 중인 여성 근로자가 모성을 보호하기 위하여, 또는 만 8세 이하 또는 초등학교 2학년 이하 자녀를 양육하기 위하여 휴직을 신청하는 경우 이를 허용하여야 합니다. 이는 성별에 관계없이 자녀를 양육하는 근로자가 육아휴직을 신청할 수 있다는 의미입니다. \n\n또한, 남성 근로자가 배우자의 출산을 이유로 휴가(배우자 출산휴가)를 사용하는 것도 가능하며, 제18조의2에 따라 배우자 출산휴가를 20일 동안 유급으로 사용할 수 있습니다. \n\n즉, 육아휴직은 주로 여성에게 권장되지만, 법률상 남성 근로자가 자녀 양육을 위해 신청할 수 있는 제도적 권리가 명시되어 있습니다.\n\n관련 조항:\n- 제19조(육아휴직) : "사업주는 임신 중인 여성 근로자가 모성을 보호하거나 근로자가 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위하여 휴직을 신청하는 경우에 이를 허용하여야 한다."', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_17d1f82c3e'}, id='run--9ca7a253-7eaf-4afc-b60c-161d273da5fd-0')])

In [60]:
len(store)


2